In [1]:
import pandas as pd
import numpy as np
from fastai.vision.all import *

In [2]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 50)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
 #Enter desired years of data
YEARS = [2020]

data = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

In [5]:
data['uid'] = data['game_id'] \
                + '-' + data['drive'].fillna(0).astype(int).astype(str) \
                + '-' + data['play_id'].astype(str)

In [6]:
data['play_id']

0          1
1         39
2         54
3         93
4        118
        ... 
8757    4016
8758    4035
8759    4050
8760    4071
8761    4098
Name: play_id, Length: 8762, dtype: int64

In [7]:
data['drive'].fillna(0).astype(int).astype(str)

0        0
1        1
2        1
3        1
4        1
        ..
8757    22
8758    22
8759    23
8760    23
8761    23
Name: drive, Length: 8762, dtype: object

In [8]:
data[['uid','home_team','away_team','posteam','qtr','time'
                         ,'total_home_score','total_away_score'
          ,'down','ydstogo','yardline_100','ydsnet','play_type','yards_gained','desc','penalty','penalty_type','penalty_yards',
     'play_id','game_id','drive','old_game_id','shotgun']]\
.query('(away_team=="DAL")&'
       '(qtr==3) & (time=="13:34")' ).head(5)

,uid,home_team,away_team,posteam,qtr,time,total_home_score,total_away_score,down,ydstogo,yardline_100,ydsnet,play_type,yards_gained,desc,penalty,penalty_type,penalty_yards,play_id,game_id,drive,old_game_id,shotgun
658,2020_01_DAL_LA-10-2468,LA,DAL,DAL,3,13:34,13,14,3.0,10,56.0,21.0,pass,11.0,(13:34) (Shotgun) 4-D.Prescott pass short middle to 86-D.Schultz to LA 45 for 11 yards (22-T.Hill).,0.0,NaN,NaN,2468,2020_01_DAL_LA,10.0,2020091312,1


In [9]:
import os

In [10]:
image_files = os.listdir('images')

In [11]:
image_play_indeces = [os.path.splitext(i)[0] for i in image_files]

In [12]:
image_play_indeces

['2020_01_DAL_LA-1-122',
 '2020_01_DAL_LA-1-196',
 '2020_01_DAL_LA-1-296',
 '2020_01_DAL_LA-1-77',
 '2020_01_DAL_LA-10-2404',
 '2020_01_DAL_LA-10-2468',
 '2020_01_DAL_LA-2-351',
 '2020_01_DAL_LA-2-417',
 '2020_01_DAL_LA-3-560',
 '2020_01_DAL_LA-3-648',
 '2020_01_DAL_LA-3-672',
 '2020_01_DAL_LA-3-717',
 '2020_01_DAL_LA-8-2241',
 '2020_02_BUF_MIA-1-122',
 '2020_02_BUF_MIA-1-41',
 '2020_02_BUF_MIA-18-3218',
 '2020_02_BUF_MIA-18-3242',
 '2020_02_BUF_MIA-18-3341',
 '2020_02_BUF_MIA-19-3396',
 '2020_02_BUF_MIA-19-3455',
 '2020_02_BUF_MIA-2-193',
 '2020_02_BUF_MIA-2-260',
 '2020_02_BUF_MIA-3-327',
 '2020_02_BUF_MIA-3-395',
 '2020_02_BUF_MIA-3-419',
 '2020_02_BUF_MIA-3-440',
 '2020_02_BUF_MIA-3-511',
 '2020_02_BUF_MIA-3-546',
 '2020_02_BUF_MIA-4-602',
 '2020_02_BUF_MIA-4-718',
 '2020_02_BUF_MIA-4-766',
 '2020_02_BUF_MIA-9-1456',
 '2020_02_BUF_MIA-9-1480',
 '2020_02_BUF_MIA-9-1673']

target_image is basically to define whether a play was shotgun or not.

In [33]:
target_image = data[data.uid.isin(image_play_indeces)][['shotgun','uid']]

In [15]:
def shotgun_labels(f): return target_image.loc[f].shotgun

In [16]:
shotgun_labels(image_play_indeces[5])

1

In [46]:
target_image['fn'] = target_image.uid + '.png'

In [18]:
data.query("uid=='2020_01_DAL_LA-1-122'")

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,away_coach,away_score,away_team,away_timeouts_remaining,away_wp,away_wp_post,blocked_player_id,blocked_player_name,comp_air_epa,comp_air_wpa,comp_yac_epa,comp_yac_wpa,complete_pass,cp,cpoe,def_wp,defensive_extra_point_attempt,defensive_extra_point_conv,defensive_two_point_attempt,defensive_two_point_conv,defteam,defteam_score,defteam_score_post,defteam_timeouts_remaining,desc,div_game,down,drive,drive_end_transition,drive_end_yard_line,drive_ended_with_score,drive_first_downs,drive_game_clock_end,drive_game_clock_start,drive_inside20,drive_play_count,drive_play_id_ended,drive_play_id_started,drive_quarter_end,drive_quarter_start,drive_real_start_time,drive_start_transition,drive_start_yard_line,drive_time_of_possession,drive_yards_penalized,end_clock_time,end_yard_line,ep,epa,extra_point_attempt,extra_point_prob,extra_point_result,fg_prob,field_goal_attempt,field_goal_result,first_down,first_down_pass,first_down_penalty,first_down_rush,fixed_drive,fixed_drive_result,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_1_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,forced_fumble_player_2_team,fourth_down_converted,fourth_down_failed,fumble,fumble_forced,fumble_lost,fumble_not_forced,fumble_out_of_bounds,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumbled_1_player_id,fumbled_1_player_name,fumbled_1_team,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,game_date,game_half,game_id,game_seconds_remaining,game_stadium,goal_to_go,half_seconds_remaining,home_coach,home_score,home_team,home_timeouts_remaining,home_wp,home_wp_post,id,incomplete_pass,interception,interception_player_id,interception_player_name,jersey_number,kick_distance,kicker_player_id,kicker_player_name,kickoff_attempt,kickoff_downed,kickoff_fair_catch,kickoff_in_endzone,kickoff_inside_twenty,kickoff_out_of_bounds,kickoff_returner_player_id,kickoff_returner_player_name,lateral_interception_player_id,lateral_interception_player_name,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_reception,lateral_recovery,lateral_return,lateral_rush,lateral_rusher_player_id,lateral_rusher_player_name,lateral_sack_player_id,lateral_sack_player_name,location,name,nfl_api_id,no_huddle,no_score_prob,old_game_id,opp_fg_prob,opp_safety_prob,opp_td_prob,order_sequence,own_kickoff_recovery,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,own_kickoff_recovery_td,pass,pass_attempt,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,pass_length,pass_location,pass_touchdown,passer,passer_id,passer_jersey_number,passer_player_id,passer_player_name,penalty,penalty_player_id,penalty_player_name,penalty_team,penalty_type,penalty_yards,play,play_clock,play_deleted,play_id,play_type,play_type_nfl,posteam,posteam_score,posteam_score_post,posteam_timeouts_remaining,posteam_type,punt_attempt,punt_blocked,punt_downed,punt_fair_catch,punt_in_endzone,punt_inside_twenty,punt_out_of_bounds,punt_returner_player_id,punt_returner_player_name,punter_player_id,punter_player_name,qb_dropback,qb_epa,qb_hit,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,qb_kneel,qb_scramble,qb_spike,qtr,quarter_end,quarter_seconds_remaining,receiver,receiver_id,receiver_jersey_number,receiver_player_id,re

Next thing to do is get a list of images.

In [ ]:
from os import path

In [20]:
from pathlib import Path

In [37]:
p = Path('./images')

In [ ]:
 x= p / 'images'

In [ ]:
path2 = untar_data(URLs.MNIST_TINY)

In [28]:
target_image.reset_index(inplace=True)

In [47]:
target_image

,shotgun,uid,fn
559,0,2020_01_DAL_LA-1-77,2020_01_DAL_LA-1-77.png
561,0,2020_01_DAL_LA-1-122,2020_01_DAL_LA-1-122.png
564,0,2020_01_DAL_LA-1-196,2020_01_DAL_LA-1-196.png
567,0,2020_01_DAL_LA-1-296,2020_01_DAL_LA-1-296.png
570,0,2020_01_DAL_LA-2-351,2020_01_DAL_LA-2-351.png
573,1,2020_01_DAL_LA-2-417,2020_01_DAL_LA-2-417.png
579,1,2020_01_DAL_LA-3-560,2020_01_DAL_LA-3-560.png
583,1,2020_01_DAL_LA-3-648,2020_01_DAL_LA-3-648.png
584,1,2020_01_DAL_LA-3-672,2020_01_DAL_LA-3-672.png
586,0,2020_01_DAL_LA-3-717,2020_01_DAL_LA-3-717.png


In [38]:
p

Path('images')

In [49]:


ImageDataLoaders.from_df(target_image,folder='images', fn_col=2,label_col=0)